In [1]:
from energy_model.models.persistence_manager import PersistenceManager
from energy_model.models.system_energy_model import SystemEnergyModel
from energy_model.energy_model_parameters import FULL_DATASET_BEFORE_PROCESSING_PATH, SYSTEM_ENERGY_MODEL_FILE_NAME
import pandas as pd

full_df = pd.read_csv(FULL_DATASET_BEFORE_PROCESSING_PATH, index_col=0)

system_model = SystemEnergyModel()
system_model.build_energy_model(full_df)
PersistenceManager.save_model(system_model, SYSTEM_ENERGY_MODEL_FILE_NAME)


--- Running Fold 1/5 ---
*** Model's Accuracy on test-set ***
Symmetric Mean Absolute Percentage Error (Symmetric MAPE) value: 13.64162302179975
Relative Root Mean Squared Error (Relative RMSE) value: 10.406659708970542
Standard Deviation (Std.) value: 24.608806169901964
Average Percentage Error Rate (Average PER) value: 11.496411458638105
Mean Squared Error (MSE) value: 11.280763593215953
Root Mean Squared Error (RMSE) value: 3.358684801111285
Mean Absolute Error (MAE) value: 2.143883083107851
Root Mean Squared Percent Error (RMSPE) value: 33.52010591144074
New best model found on fold 1 with Symmetric Mean Absolute Percentage Error (Symmetric MAPE)=13.64162302179975

--- Running Fold 2/5 ---
*** Model's Accuracy on test-set ***
Symmetric Mean Absolute Percentage Error (Symmetric MAPE) value: 15.843199989992952
Relative Root Mean Squared Error (Relative RMSE) value: 10.710583257088166
Standard Deviation (Std.) value: 22.829397817104358
Average Percentage Error Rate (Average PER) valu

In [7]:
test_set = pd.read_csv("test_energy_consumption_system_mWh.csv")

In [8]:
test_set.head(5)

,duration,cpu_seconds_system,memory_mb_relative_system,disk_read_kb_system,disk_read_count_system,disk_write_kb_system,disk_write_count_system,disk_read_time_ms_system,disk_write_time_ms_system,network_kb_sent_system,network_packets_sent_system,network_kb_received_system,network_packets_received_system,target
0,5.260686,4.192767,30.720,692.0,37,6674.5,408,0,0,0.700,8,17,17,10.924691
1,5.307450,2.948288,2.048,160.0,13,1307.5,101,0,0,0.000,0,17,17,11.021805
2,5.279081,3.127855,-7.168,16.0,1,628.0,29,0,0,0.000,0,2,2,10.962892
3,5.355309,2.913288,15.360,8484.0,466,8270.0,125,1,0,6.740,83,4,4,11.121192
4,5.247246,2.878114,-4.096,264.0,14,2273.0,191,0,0,2.076,25,21,21,10.896781


In [9]:
y_real = test_set['target']

In [10]:
x_test = test_set.drop('target', axis=1)

In [11]:
y_pred = system_model.predict(x_test)

In [12]:
len(y_pred)

3253

In [13]:
len(y_real)

3253

In [14]:
diff = y_pred - y_real

In [17]:
all_results = {}
for energy_process in [0.5, 1, 2.5, 5, 10]:
    all_results[energy_process] = len([d for d in diff if d > energy_process])

In [18]:
all_results


{0.5: 1369, 1: 1127, 2.5: 514, 5: 144, 10: 16}

In [23]:
import numpy as np


def percentile_absolute_error(y_true, y_pred, percentile=95):
    """
    Computes the percentile of absolute errors.

    Args:
        y_true (array-like): Ground truth values
        y_pred (array-like): Predicted values
        percentile (float): Percentile to compute (0-100)

    Returns:
        float: Percentile absolute error
    """
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)

    squared_errors = (y_true - y_pred) ** 2
    return np.percentile(squared_errors, percentile)

In [24]:
percentile_absolute_error(y_real, y_pred)

np.float64(38.254886728612014)

In [ ]:
huber_system_model = SystemEnergyModel()
huber_system_model.build_energy_model(full_df)